In [1]:
# ===================================================================
# PART 0: 라이브러리 임포트
# ===================================================================
import numpy as np
import pandas as pd
import os
import joblib
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

print(f"TensorFlow Version: {tf.__version__}")


TensorFlow Version: 2.12.0


In [6]:

# ===================================================================
# PART 1: 경로 및 하이퍼파라미터 설정
# ===================================================================
# --- 입력 데이터 경로 ---
IMAGE_DIRECTORY = '../data/interim/satellites'
TABULAR_DATA_PATH = '../data/interim/gang_nam_sendimental_score_with_sale.csv'

# --- 출력 데이터 및 모델 저장 경로 ---
# 저장 경로가 없으면 자동으로 생성
os.makedirs('../data/final', exist_ok=True)
os.makedirs('../models', exist_ok=True)

# 각 데이터셋(train, val, test)별로 벡터 파일을 저장할 경로 템플릿
IMAGE_VECTOR_SAVE_PATH = '../data/final/image_features_{split}_32d.npy'
TABULAR_VECTOR_SAVE_PATH = '../data/final/tabular_features_{split}_64d.npy'

# 특징 추출에 사용된 모델(PCA, Scaler) 저장 경로
PCA_MODEL_SAVE_PATH = '../models/pca_image_feature_extractor.pkl'
SCALER_SAVE_PATH = '../models/tabular_minmax_scaler.pkl'

# --- 하이퍼파라미터 ---
IMAGE_PCA_COMPONENTS = 32
TABULAR_MLP_COMPONENTS = 64
BATCH_SIZE = 32


In [7]:

# ===================================================================
# PART 2: 데이터 로딩 및 분할 (Data Loading & Splitting)
# 모든 전처리 이전에 데이터를 시간순으로 분할하여 데이터 유출(Data Leakage)을 방지합니다.
# ===================================================================
print("\nPART 2: 데이터 로딩 및 분할 시작...")
df = pd.read_csv(TABULAR_DATA_PATH)
df['계약일자'] = pd.to_datetime(df['계약일자'])
df = df.sort_values('계약일자').reset_index(drop=True)

# 이미지 경로 생성
df['image_filename'] = [f'apt_image_{i}.jpg' for i in df.index]
df['image_path'] = df['image_filename'].apply(lambda f: os.path.join(IMAGE_DIRECTORY, f))

# 데이터 분할 (7:2:1 비율)
n_total = len(df)
train_end = int(n_total * 0.7)
val_end = int(n_total * 0.9)

train_df = df[:train_end].copy()
val_df = df[train_end:val_end].copy()
test_df = df[val_end:].copy()

print(f"데이터 분할 완료: Train={len(train_df)}, Validation={len(val_df)}, Test={len(test_df)}")
print("PART 2: 완료!")



PART 2: 데이터 로딩 및 분할 시작...
데이터 분할 완료: Train=15092, Validation=4312, Test=2157
PART 2: 완료!


In [8]:

# ===================================================================
# PART 3: 이미지 특징 추출 (Image Feature Extraction)
# 중요: PCA 모델은 훈련 데이터(train_df)에만 `fit`하고, 모든 데이터셋에 `transform`을 적용합니다.
# ===================================================================
print("\nPART 3: 이미지 특징 추출 시작...")

def extract_image_features_2048d(image_paths, batch_size=BATCH_SIZE):
    """ResNet50으로 고차원 특징 벡터를 추출하는 헬퍼 함수"""
    image_input = Input(shape=(224, 224, 3), name='image_input')
    base_cnn = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_tensor=image_input)
    base_cnn.trainable = False
    extractor_model = Model(inputs=image_input, outputs=base_cnn.output)
    
    def image_generator(paths, b_size):
        for i in range(0, len(paths), b_size):
            batch_paths = paths[i:i+b_size]
            batch_images = []
            for p in batch_paths:
                try:
                    img = load_img(p, target_size=(224, 224))
                    img_array = img_to_array(img)
                    batch_images.append(img_array)
                except (FileNotFoundError, IOError):
                    print(f"경고: {p} 이미지를 찾을 수 없습니다. 0으로 대체합니다.")
                    batch_images.append(np.zeros((224, 224, 3)))
            yield preprocess_input(np.array(batch_images))
            
    num_batches = int(np.ceil(len(image_paths) / batch_size))
    features = extractor_model.predict(image_generator(image_paths, batch_size), steps=num_batches, verbose=1)
    return features

# 1. 훈련 데이터의 이미지로 PCA 모델 학습(fit)
print("훈련 데이터로 PCA 모델 학습...")
train_image_features_2048d = extract_image_features_2048d(train_df['image_path'].tolist())
pca = PCA(n_components=IMAGE_PCA_COMPONENTS)
train_image_features_32d = pca.fit_transform(train_image_features_2048d)
print(f"PCA 설명된 분산 비율: {np.sum(pca.explained_variance_ratio_):.4f}")

# 2. 학습된 PCA 모델로 검증 및 테스트 데이터 변환(transform)
print("검증 및 테스트 데이터 변환...")
val_image_features_2048d = extract_image_features_2048d(val_df['image_path'].tolist())
val_image_features_32d = pca.transform(val_image_features_2048d)

test_image_features_2048d = extract_image_features_2048d(test_df['image_path'].tolist())
test_image_features_32d = pca.transform(test_image_features_2048d)

# 3. 특징 벡터 및 PCA 모델 저장
np.save(IMAGE_VECTOR_SAVE_PATH.format(split='train'), train_image_features_32d)
np.save(IMAGE_VECTOR_SAVE_PATH.format(split='val'), val_image_features_32d)
np.save(IMAGE_VECTOR_SAVE_PATH.format(split='test'), test_image_features_32d)
joblib.dump(pca, PCA_MODEL_SAVE_PATH)
print(f"이미지 특징 벡터 3개와 PCA 모델이 '{os.path.dirname(PCA_MODEL_SAVE_PATH)}' 및 '{os.path.dirname(IMAGE_VECTOR_SAVE_PATH)}'에 저장되었습니다.")
print("PART 3: 완료!")



PART 3: 이미지 특징 추출 시작...
훈련 데이터로 PCA 모델 학습...


2025-10-10 16:46:10.985233: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


472/472 [==============================] - 404s 854ms/step
PCA 설명된 분산 비율: 0.7215
검증 및 테스트 데이터 변환...
68/68 [==============================] - 52s 763ms/step
이미지 특징 벡터 3개와 PCA 모델이 '../models' 및 '../data/final'에 저장되었습니다.
PART 3: 완료!


In [9]:

# ===================================================================
# PART 4: 정형 데이터 특징 추출 (Tabular Feature Extraction)
# 중요: MinMaxScaler는 훈련 데이터에만 `fit`하고, 모든 데이터셋에 `transform`을 적용합니다.
# ===================================================================
print("\nPART 4: 정형 데이터 특징 추출 시작...")

tabular_cols = [
    '전용면적(㎡)', '층', '건축년도', '아파트 나이', 'leading_index', '건설기성액(백만원)', 
    '부동산_소비심리지수', '주택시장_소비심리지수', '강남구_변동률', '강남구_누계', 
    '토지시장_소비심리지수', '아파트_호수', '아파트_면적', 'rate'
]

# 1. 훈련 데이터로 스케일러 학습(fit)
scaler = MinMaxScaler()
train_tabular_scaled = scaler.fit_transform(train_df[tabular_cols])

# 2. 학습된 스케일러로 검증 및 테스트 데이터 변환(transform)
val_tabular_scaled = scaler.transform(val_df[tabular_cols])
test_tabular_scaled = scaler.transform(test_df[tabular_cols])

# 3. MLP 특징 추출기 모델 생성
mlp_input = Input(shape=(len(tabular_cols),), name='tabular_input')
embedding_layer = Dense(TABULAR_MLP_COMPONENTS, activation='relu')(mlp_input)
mlp_extractor = Model(inputs=mlp_input, outputs=embedding_layer)

# 4. 각 데이터셋을 MLP로 변환
print("MLP를 이용해 정형 데이터 특징 추출 중...")
train_tabular_features_64d = mlp_extractor.predict(train_tabular_scaled)
val_tabular_features_64d = mlp_extractor.predict(val_tabular_scaled)
test_tabular_features_64d = mlp_extractor.predict(test_tabular_scaled)

# 5. 특징 벡터 및 스케일러 저장
np.save(TABULAR_VECTOR_SAVE_PATH.format(split='train'), train_tabular_features_64d)
np.save(TABULAR_VECTOR_SAVE_PATH.format(split='val'), val_tabular_features_64d)
np.save(TABULAR_VECTOR_SAVE_PATH.format(split='test'), test_tabular_features_64d)
joblib.dump(scaler, SCALER_SAVE_PATH)
print(f"정형 데이터 특징 벡터 3개와 스케일러가 '{os.path.dirname(SCALER_SAVE_PATH)}' 및 '{os.path.dirname(TABULAR_VECTOR_SAVE_PATH)}'에 저장되었습니다.")
print("PART 4: 완료!")

print("\n🎉 모든 데이터 준비 과정이 완료되었습니다.")



PART 4: 정형 데이터 특징 추출 시작...
MLP를 이용해 정형 데이터 특징 추출 중...
68/68 [==============================] - 0s 168us/step
정형 데이터 특징 벡터 3개와 스케일러가 '../models' 및 '../data/final'에 저장되었습니다.
PART 4: 완료!

🎉 모든 데이터 준비 과정이 완료되었습니다.
